上一章从零开始的实现，大部分操作在`PyTorch`中都有现成的组件。

## 1.生成数据集

In [20]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 2.读取数据集

In [21]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [22]:
next(iter(data_iter))

[tensor([[ 1.1030e+00,  1.2113e+00],
         [-1.9628e+00, -1.5389e+00],
         [ 1.1084e+00, -1.4502e+00],
         [ 8.0156e-01,  3.1062e-01],
         [ 4.8958e-01, -6.0294e-01],
         [ 1.9926e-01,  5.1988e-01],
         [-2.0370e+00,  1.3557e+00],
         [-2.6732e-04, -1.1384e+00],
         [ 1.8473e+00,  1.4467e+00],
         [ 1.4747e+00,  7.4421e-01]]),
 tensor([[ 2.2916],
         [ 5.5219],
         [11.3455],
         [ 4.7644],
         [ 7.2243],
         [ 2.8308],
         [-4.4660],
         [ 8.0637],
         [ 2.9657],
         [ 4.6176]])]

## 3.定义模型
在 `PyTorch`中，全连接层在 `Linear` 类中定义。
值得注意的是，我们将两个参数传递到 `nn.Linear` 中。
- 第一个指定输入特征形状，即2，
- 第二个指定输出特征形状，输出特征形状为单个标量，因此为1。

In [23]:
from torch import nn

net = nn.Sequential(nn.Linear(2,1))

## 4. 初始化模型参数
`PyTorch` 通常有预定义的方法来初始化参数。
 本例指定每个权重参数应该从**均值为0、标准差为0.01的正态分布**中随机采样，偏置参数将初始化为 0 。

In [24]:
# PyTorch 允许直接通过 weight.data 和 bias.data 访问参数
# 超参数位于第一个图层，即 net[0]
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 5.定义损失函数

In [25]:
loss = nn.MSELoss()

## 6.定义优化算法
小批量随机梯度下降算法是一种优化神经网络的标准工具， PyTorch在optim模块中实现了该算法的许多变种。
小批量随机梯度下降只需要设置lr值，这里设置为0.03

In [26]:
trainer = torch.optim.SGD(net.parameters(),lr=0.03)

# 7.训练

In [27]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000285
epoch 2, loss 0.000100
epoch 3, loss 0.000100


## 8.估计误差

In [28]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([0.0003, 0.0003])
b的估计误差： tensor([0.0009])
